# Music Generation Using Deep Learning

## Real World Problem

This case-study focuses on generating music automatically using Recurrent Neural Network(RNN).<br> 
We do not necessarily have to be a music expert in order to generate music. Even a non expert can generate a decent quality music using RNN.<br>
We all like to listen interesting music and if there is some way to generate music automatically, particularly decent quality music then it's a big leap in the world of music industry.<br><br>
<b>Task:</b> Our task here is to take some existing music data then train a model using this existing data. The model has to learn the patterns in music that we humans enjoy. Once it learns this, the model should be able to generate new music for us. It cannot simply copy-paste from the training data. It has to understand the patterns of music to generate new music. We here are not expecting our model to generate new music which is of professional quality, but we want it to generate a decent quality music which should be melodious and good to hear.<br><br>
Now, what is music? In short music is nothing but a sequence of musical notes. Our input to the model is a sequence of musical events/notes. Our output will be new sequence of musical events/notes. In this case-study we have limited our self to single instrument music as this is our first cut model. In future, we will extend this to multiple instrument music. 

## Data Source:
1. http://abc.sourceforge.net/NMD/
2. http://trillian.mit.edu/~jc/music/book/oneills/1850/X/

### From first data-source, we have downloaded first two files:
* Jigs (340 tunes)
* Hornpipes (65 tunes)

In [1]:
import os
import json
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding

Using TensorFlow backend.


In [2]:
data_directory = "../Data/"
data_file = "Data_Tunes.txt"
charIndex_json = "char_to_index.json"
model_weights_directory = '../Data/Model_Weights/'
BATCH_SIZE = 16
SEQ_LENGTH = 64

In [3]:
def read_batches(all_chars, unique_chars):
    length = all_chars.shape[0]
    batch_chars = int(length / BATCH_SIZE) #155222/16 = 9701
    
    for start in range(0, batch_chars - SEQ_LENGTH, 64):  #(0, 9637, 64)  #it denotes number of batches. It runs everytime when
        #new batch is created. We have a total of 151 batches.
        X = np.zeros((BATCH_SIZE, SEQ_LENGTH))    #(16, 64)
        Y = np.zeros((BATCH_SIZE, SEQ_LENGTH, unique_chars))   #(16, 64, 87)
        for batch_index in range(0, 16):  #it denotes each row in a batch.  
            for i in range(0, 64):  #it denotes each column in a batch. Each column represents each character means 
                #each time-step character in a sequence.
                X[batch_index, i] = all_chars[batch_index * batch_chars + start + i]
                Y[batch_index, i, all_chars[batch_index * batch_chars + start + i + 1]] = 1 #here we have added '1' because the
                #correct label will be the next character in the sequence. So, the next character will be denoted by
                #all_chars[batch_index * batch_chars + start + i + 1]
        yield X, Y

In [4]:
def built_model(batch_size, seq_length, unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (batch_size, seq_length))) 
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(128, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(TimeDistributed(Dense(unique_chars)))

    model.add(Activation("softmax"))
    
    return model

In [5]:
def training_model(data, epochs = 80):
    #mapping character to index
    char_to_index = {ch: i for (i, ch) in enumerate(sorted(list(set(data))))}
    print("Number of unique characters in our whole tunes database = {}".format(len(char_to_index))) #87
    
    with open(os.path.join(data_directory, charIndex_json), mode = "w") as f:
        json.dump(char_to_index, f)
        
    index_to_char = {i: ch for (ch, i) in char_to_index.items()}
    unique_chars = len(char_to_index)
    
    model = built_model(BATCH_SIZE, SEQ_LENGTH, unique_chars)
    model.summary()
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    
    all_characters = np.asarray([char_to_index[c] for c in data], dtype = np.int32)
    print("Total number of characters = "+str(all_characters.shape[0])) #155222
    
    epoch_number, loss, accuracy = [], [], []
    
    for epoch in range(epochs):
        print("Epoch {}/{}".format(epoch+1, epochs))
        final_epoch_loss, final_epoch_accuracy = 0, 0
        epoch_number.append(epoch+1)
        
        for i, (x, y) in enumerate(read_batches(all_characters, unique_chars)):
            final_epoch_loss, final_epoch_accuracy = model.train_on_batch(x, y) #check documentation of train_on_batch here: https://keras.io/models/sequential/
            print("Batch: {}, Loss: {}, Accuracy: {}".format(i+1, final_epoch_loss, final_epoch_accuracy))
            #here, above we are reading the batches one-by-one and train our model on each batch one-by-one.
        loss.append(final_epoch_loss)
        accuracy.append(final_epoch_accuracy)
        
        #saving weights after every 10 epochs
        if (epoch + 1) % 10 == 0:
            if not os.path.exists(model_weights_directory):
                os.makedirs(model_weights_directory)
            model.save_weights(os.path.join(model_weights_directory, "Weights_{}.h5".format(epoch+1)))
            print('Saved Weights at epoch {} to file Weights_{}.h5'.format(epoch+1, epoch+1))
    
    #creating dataframe and record all the losses and accuracies at each epoch
    log_frame = pd.DataFrame(columns = ["Epoch", "Loss", "Accuracy"])
    log_frame["Epoch"] = epoch_number
    log_frame["Loss"] = loss
    log_frame["Accuracy"] = accuracy
    log_frame.to_csv("../Data/log.csv", index = False)

In [6]:
file = open(os.path.join(data_directory, data_file), mode = 'r')
data = file.read()
file.close()
if __name__ == "__main__":
    training_model(data)

Number of unique characters in our whole tunes database = 87
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (16, 64, 512)             44544     
_________________________________________________________________
lstm_1 (LSTM)                (16, 64, 256)             787456    
_________________________________________________________________
dropout_1 (Dropout)          (16, 64, 256)             0         
_________________________________________________________________
lstm_2 (LSTM)                (16, 64, 128)             197120    
_________________________________________________________________
dropout_2 (Dropout)          (16, 64, 128)             0         
_________________________________________________________________
time_distributed_1 (TimeDist (16, 64, 87)              11223     
_________________________________________________________________
activation_1 (A

Batch: 118, Loss: 2.957672119140625, Accuracy: 0.2470703125
Batch: 119, Loss: 2.94547700881958, Accuracy: 0.23828125
Batch: 120, Loss: 2.667551040649414, Accuracy: 0.283203125
Batch: 121, Loss: 2.697852611541748, Accuracy: 0.2939453125
Batch: 122, Loss: 2.977010488510132, Accuracy: 0.2373046875
Batch: 123, Loss: 2.888855218887329, Accuracy: 0.2548828125
Batch: 124, Loss: 2.84230899810791, Accuracy: 0.255859375
Batch: 125, Loss: 2.7291135787963867, Accuracy: 0.2734375
Batch: 126, Loss: 2.714336395263672, Accuracy: 0.2724609375
Batch: 127, Loss: 2.8596792221069336, Accuracy: 0.255859375
Batch: 128, Loss: 2.8016529083251953, Accuracy: 0.271484375
Batch: 129, Loss: 2.747270107269287, Accuracy: 0.2734375
Batch: 130, Loss: 2.761723518371582, Accuracy: 0.26171875
Batch: 131, Loss: 2.7729458808898926, Accuracy: 0.2724609375
Batch: 132, Loss: 2.8464207649230957, Accuracy: 0.24609375
Batch: 133, Loss: 2.738624095916748, Accuracy: 0.2783203125
Batch: 134, Loss: 2.674128532409668, Accuracy: 0.2744

Batch: 106, Loss: 2.0507140159606934, Accuracy: 0.439453125
Batch: 107, Loss: 2.094038248062134, Accuracy: 0.4248046875
Batch: 108, Loss: 2.140477180480957, Accuracy: 0.4462890625
Batch: 109, Loss: 2.0874061584472656, Accuracy: 0.4443359375
Batch: 110, Loss: 1.88535737991333, Accuracy: 0.4814453125
Batch: 111, Loss: 1.929410457611084, Accuracy: 0.46484375
Batch: 112, Loss: 2.1033430099487305, Accuracy: 0.4541015625
Batch: 113, Loss: 2.1319150924682617, Accuracy: 0.4384765625
Batch: 114, Loss: 2.051302671432495, Accuracy: 0.4345703125
Batch: 115, Loss: 2.061023473739624, Accuracy: 0.4462890625
Batch: 116, Loss: 2.076653242111206, Accuracy: 0.4189453125
Batch: 117, Loss: 2.163181781768799, Accuracy: 0.4287109375
Batch: 118, Loss: 2.000194787979126, Accuracy: 0.48046875
Batch: 119, Loss: 2.0290579795837402, Accuracy: 0.466796875
Batch: 120, Loss: 1.984151840209961, Accuracy: 0.453125
Batch: 121, Loss: 1.9987012147903442, Accuracy: 0.458984375
Batch: 122, Loss: 2.07409405708313, Accuracy: 

Batch: 94, Loss: 1.6274139881134033, Accuracy: 0.517578125
Batch: 95, Loss: 1.6308321952819824, Accuracy: 0.5
Batch: 96, Loss: 1.710994839668274, Accuracy: 0.505859375
Batch: 97, Loss: 1.6148775815963745, Accuracy: 0.53125
Batch: 98, Loss: 1.6052021980285645, Accuracy: 0.5546875
Batch: 99, Loss: 1.5704319477081299, Accuracy: 0.53125
Batch: 100, Loss: 1.603724479675293, Accuracy: 0.5185546875
Batch: 101, Loss: 1.6103531122207642, Accuracy: 0.515625
Batch: 102, Loss: 1.5851283073425293, Accuracy: 0.5244140625
Batch: 103, Loss: 1.7265279293060303, Accuracy: 0.5380859375
Batch: 104, Loss: 1.5687236785888672, Accuracy: 0.541015625
Batch: 105, Loss: 1.684617280960083, Accuracy: 0.4931640625
Batch: 106, Loss: 1.7031844854354858, Accuracy: 0.4990234375
Batch: 107, Loss: 1.7944755554199219, Accuracy: 0.4853515625
Batch: 108, Loss: 1.8079833984375, Accuracy: 0.4873046875
Batch: 109, Loss: 1.7943141460418701, Accuracy: 0.4873046875
Batch: 110, Loss: 1.542128562927246, Accuracy: 0.546875
Batch: 11

Batch: 82, Loss: 1.5306901931762695, Accuracy: 0.5126953125
Batch: 83, Loss: 1.425638198852539, Accuracy: 0.5810546875
Batch: 84, Loss: 1.4679248332977295, Accuracy: 0.5869140625
Batch: 85, Loss: 1.4591352939605713, Accuracy: 0.5712890625
Batch: 86, Loss: 1.6338322162628174, Accuracy: 0.509765625
Batch: 87, Loss: 1.4532408714294434, Accuracy: 0.587890625
Batch: 88, Loss: 1.5881534814834595, Accuracy: 0.5283203125
Batch: 89, Loss: 1.5803816318511963, Accuracy: 0.515625
Batch: 90, Loss: 1.4890458583831787, Accuracy: 0.5439453125
Batch: 91, Loss: 1.4976420402526855, Accuracy: 0.5576171875
Batch: 92, Loss: 1.586625099182129, Accuracy: 0.5234375
Batch: 93, Loss: 1.4662755727767944, Accuracy: 0.55859375
Batch: 94, Loss: 1.4694645404815674, Accuracy: 0.556640625
Batch: 95, Loss: 1.4774346351623535, Accuracy: 0.529296875
Batch: 96, Loss: 1.5403748750686646, Accuracy: 0.541015625
Batch: 97, Loss: 1.393653392791748, Accuracy: 0.5771484375
Batch: 98, Loss: 1.4362659454345703, Accuracy: 0.5859375


...

Batch: 32, Loss: 0.6080125570297241, Accuracy: 0.7919921875
Batch: 33, Loss: 0.7245921492576599, Accuracy: 0.767578125
Batch: 34, Loss: 0.7593650221824646, Accuracy: 0.7451171875
Batch: 35, Loss: 0.6916241645812988, Accuracy: 0.7646484375
Batch: 36, Loss: 0.6882265210151672, Accuracy: 0.7890625
Batch: 37, Loss: 0.6773490309715271, Accuracy: 0.7763671875
Batch: 38, Loss: 0.6676771640777588, Accuracy: 0.7763671875
Batch: 39, Loss: 0.7009940147399902, Accuracy: 0.7666015625
Batch: 40, Loss: 0.6422684192657471, Accuracy: 0.791015625
Batch: 41, Loss: 0.6108037233352661, Accuracy: 0.7919921875
Batch: 42, Loss: 0.5233389139175415, Accuracy: 0.8271484375
Batch: 43, Loss: 0.6658971309661865, Accuracy: 0.7734375
Batch: 44, Loss: 0.6941579580307007, Accuracy: 0.765625
Batch: 45, Loss: 0.6287591457366943, Accuracy: 0.78515625
Batch: 46, Loss: 0.5957440733909607, Accuracy: 0.798828125
Batch: 47, Loss: 0.6154074668884277, Accuracy: 0.810546875
Batch: 48, Loss: 0.6166465282440186, Accuracy: 0.7851562

Batch: 20, Loss: 0.5763198137283325, Accuracy: 0.806640625
Batch: 21, Loss: 0.6234312653541565, Accuracy: 0.7919921875
Batch: 22, Loss: 0.7313104271888733, Accuracy: 0.763671875
Batch: 23, Loss: 0.6582440137863159, Accuracy: 0.779296875
Batch: 24, Loss: 0.703904390335083, Accuracy: 0.767578125
Batch: 25, Loss: 0.6492465138435364, Accuracy: 0.7900390625
Batch: 26, Loss: 0.5761775970458984, Accuracy: 0.8076171875
Batch: 27, Loss: 0.6412630081176758, Accuracy: 0.771484375
Batch: 28, Loss: 0.6744866371154785, Accuracy: 0.755859375
Batch: 29, Loss: 0.6236093044281006, Accuracy: 0.7958984375
Batch: 30, Loss: 0.5587791204452515, Accuracy: 0.814453125
Batch: 31, Loss: 0.5736687183380127, Accuracy: 0.8125
Batch: 32, Loss: 0.5977579355239868, Accuracy: 0.7978515625
Batch: 33, Loss: 0.6908916234970093, Accuracy: 0.7705078125
Batch: 34, Loss: 0.7303498983383179, Accuracy: 0.7763671875
Batch: 35, Loss: 0.6518499851226807, Accuracy: 0.7734375
Batch: 36, Loss: 0.6948391795158386, Accuracy: 0.77929687

Batch: 8, Loss: 0.6415903568267822, Accuracy: 0.78515625
Batch: 9, Loss: 0.6614282131195068, Accuracy: 0.7861328125
Batch: 10, Loss: 0.629571795463562, Accuracy: 0.787109375
Batch: 11, Loss: 0.7285956740379333, Accuracy: 0.763671875
Batch: 12, Loss: 0.7285448312759399, Accuracy: 0.7568359375
Batch: 13, Loss: 0.561111569404602, Accuracy: 0.8095703125
Batch: 14, Loss: 0.7494210004806519, Accuracy: 0.7490234375
Batch: 15, Loss: 0.5898087024688721, Accuracy: 0.7998046875
Batch: 16, Loss: 0.6777321696281433, Accuracy: 0.7802734375
Batch: 17, Loss: 0.6892622709274292, Accuracy: 0.7724609375
Batch: 18, Loss: 0.7134425640106201, Accuracy: 0.75
Batch: 19, Loss: 0.6523587703704834, Accuracy: 0.779296875
Batch: 20, Loss: 0.5829240083694458, Accuracy: 0.8232421875
Batch: 21, Loss: 0.6076602935791016, Accuracy: 0.8017578125
Batch: 22, Loss: 0.7249583005905151, Accuracy: 0.7734375
Batch: 23, Loss: 0.6566071510314941, Accuracy: 0.7880859375
Batch: 24, Loss: 0.6809499263763428, Accuracy: 0.76953125
Ba

Batch: 147, Loss: 0.652895450592041, Accuracy: 0.783203125
Batch: 148, Loss: 0.7141090631484985, Accuracy: 0.759765625
Batch: 149, Loss: 0.6134890913963318, Accuracy: 0.80078125
Batch: 150, Loss: 0.6529809236526489, Accuracy: 0.7763671875
Batch: 151, Loss: 0.5405412316322327, Accuracy: 0.8203125
Epoch 80/80
Batch: 1, Loss: 0.8784704208374023, Accuracy: 0.740234375
Batch: 2, Loss: 0.7270656824111938, Accuracy: 0.75
Batch: 3, Loss: 0.6270932555198669, Accuracy: 0.7880859375
Batch: 4, Loss: 0.5629788637161255, Accuracy: 0.8125
Batch: 5, Loss: 0.6740407943725586, Accuracy: 0.78125
Batch: 6, Loss: 0.663748025894165, Accuracy: 0.7880859375
Batch: 7, Loss: 0.6758555173873901, Accuracy: 0.78125
Batch: 8, Loss: 0.6195414662361145, Accuracy: 0.7734375
Batch: 9, Loss: 0.6575390100479126, Accuracy: 0.7841796875
Batch: 10, Loss: 0.6251243352890015, Accuracy: 0.794921875
Batch: 11, Loss: 0.7423152327537537, Accuracy: 0.7548828125
Batch: 12, Loss: 0.6955631971359253, Accuracy: 0.7685546875
Batch: 13,

Batch: 135, Loss: 0.5896015167236328, Accuracy: 0.810546875
Batch: 136, Loss: 0.6477150917053223, Accuracy: 0.7939453125
Batch: 137, Loss: 0.6595759391784668, Accuracy: 0.7734375
Batch: 138, Loss: 0.5945322513580322, Accuracy: 0.80078125
Batch: 139, Loss: 0.593050479888916, Accuracy: 0.78515625
Batch: 140, Loss: 0.650367021560669, Accuracy: 0.7880859375
Batch: 141, Loss: 0.698702335357666, Accuracy: 0.7607421875
Batch: 142, Loss: 0.696868896484375, Accuracy: 0.779296875
Batch: 143, Loss: 0.6461887359619141, Accuracy: 0.79296875
Batch: 144, Loss: 0.6757009029388428, Accuracy: 0.779296875
Batch: 145, Loss: 0.6117562055587769, Accuracy: 0.783203125
Batch: 146, Loss: 0.6667803525924683, Accuracy: 0.7763671875
Batch: 147, Loss: 0.6432229280471802, Accuracy: 0.7880859375
Batch: 148, Loss: 0.6984781622886658, Accuracy: 0.7646484375
Batch: 149, Loss: 0.6178109049797058, Accuracy: 0.7822265625
Batch: 150, Loss: 0.6485364437103271, Accuracy: 0.779296875
Batch: 151, Loss: 0.5648598670959473, Accu

In [7]:
log = pd.read_csv(os.path.join(data_directory, "log.csv"))
log

,Epoch,Loss,Accuracy
0,1,2.686944,0.290039
1,2,1.940787,0.484375
2,3,1.617457,0.544922
3,4,1.458334,0.561523
4,5,1.372869,0.583984
5,6,1.308803,0.602539
6,7,1.252365,0.615234
7,8,1.202019,0.619141
8,9,1.167336,0.645508
9,10,1.115653,0.657227
